In [50]:
#暑さ指数
#ソース：環境省熱中症予防サイト
#https://www.wbgt.env.go.jp/wbgt_data.php

import pandas as pd
from datetime import datetime, date
import requests


#実況値のデータを取得
yyyymm = date.today().strftime('%Y%m')
url = f'https://www.wbgt.env.go.jp/est15WG/dl/wbgt_all_{yyyymm}.csv'
wbgt = pd.read_csv(url)

#日時データ整形
wbgt.Time = wbgt.Time.str.replace('24:00','0:00')
wbgt['date_dt'] = pd.to_datetime(wbgt.Date + ' ' + wbgt.Time, format='%Y/%m/%d %H:%M')
wbgt['Date'] = pd.to_datetime(wbgt.Date)

#longフォーマットに転換
wbgt = wbgt.set_index(['Date','Time','date_dt']).rename_axis('amdno',axis=1).stack().rename('wbgt').reset_index()
wbgt.amdno = wbgt.amdno.astype(int)

#最新分のみ切り出し
##iloc[-1]などはほぼ大丈夫ですが、絶対値で指定したほうがより安全
wbgt_latest = wbgt.loc[wbgt.date_dt==wbgt.date_dt.max()]

#地点情報を取得
##観測地点の変更・追加などがあるとデータがおちてしまうので、元データを引っ張ったほうが
##少しだけ長くなりますが、jsonで軽いので
url = 'https://www.jma.go.jp/bosai/amedas/const/amedastable.json'
r = requests.get(url)
data = r.json()

locations = pd.DataFrame.from_dict(data, orient='index').reset_index()
locations = locations.rename(columns={'index':'amdno'})
locations.amdno = locations.amdno.astype(int)
#lat longを整える
locations[['lat','lon']] = locations[['lat','lon']].applymap(lambda x: x[0] + x[1]/60)

#地点情報を最新データに追加
##how='left'で元のdataframeを保持
wbgt_latest = wbgt_latest.merge(locations[['amdno','lat','lon','kjName']], on='amdno', how='left')

#暑さ指数「日常生活に関する指針」
##指針の内容をテキストでアサイン
def guideline(value):
    if value >= 31:
        return '危険'
    elif value >= 28:
        return '厳重警戒'
    elif value >= 25:
        return '警戒'
    else:
        return '注意'
wbgt_latest['guideline'] = wbgt_latest.wbgt.apply(guideline)

wbgt_latest.to_csv("./data/wbgt.csv", index=False)

In [ ]:

#df_tempdata['amdno'] = df_tempdata['amdno'].astype(int)
#url = 'https://raw.githubusercontent.com/hiroki-sakuragi/climate_change/main/amedastable.csv'
#df_location_data = pd.read_csv(url)
#df_merge = pd.merge(df_tempdata, df_location_data, on = 'amdno')
